In [ ]:
import cv2
import numpy as np
import os
from random import shuffle
import numpy as np
from keras.utils import to_categorical
from keras.models import Sequential, Model, load_model
from keras import applications
from keras import optimizers
from keras.layers import Dropout, Flatten, Dense
import tensorflow as tf
import matplotlib.pyplot as plt
import sklearn
from sklearn import metrics

In [ ]:
train_real_path="data/CEDAR_genuine.npy"
train_fake_path="data/CEDAR_forg.npy"
syn_fake_path= "data/cgan_CEDAR_forg.npy"
fake_sample=24
real_sample=24
no_contrib=55

In [ ]:
def init_model():
  with tf.device('/device:GPU:0'):
    base_model = applications.DenseNet201(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    for layer in base_model.layers: #make the layers untrainable to get only the embeddings from the model output
      layer.trainable = False

    add_model = Sequential()
    add_model.add(Flatten(input_shape=base_model.output_shape[1:]))
    add_model.add(Dense(256, activation='relu'))
    add_model.add(Dense(2, activation='sigmoid'))
    model = Model(inputs=base_model.input, outputs=add_model(base_model.output))
    model.compile(loss='binary_crossentropy', optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])
    return model

In [ ]:
with open(train_fake_path, 'rb') as f:
    fake=np.load(f)
with open(train_real_path, 'rb') as f:
    real=np.load(f)
with open(syn_fake_path, 'rb') as f:
    syn_fake=np.load(f)

In [ ]:
def data_generate(index,syn=False):
  train_features = []
  train_labels = []
  test_features = []
  test_labels = []
  skil_features = []
  skil_labels = []
  syn_fake_features = []
  syn_fake_labels = []
  syn_skil_features = []
  syn_skil_labels = []
  #labels for train real
  for j in range (0,real_sample * 2 // 3):
      train_features.append(real[index*real_sample+j])
      train_labels.append(1)
  #labels for train fake
  for j in range (0,fake_sample * 2 // 3):
      train_features.append(fake[index*fake_sample+j])
      train_labels.append(0)
  #labels for test real
  for j in range (real_sample * 2 // 3,real_sample) :
      test_features.append(real[index*real_sample+j])
      test_labels.append(1)
  #labels for test fake
  for j in range (fake_sample * 2 // 3,fake_sample) :
      test_features.append(fake[index*fake_sample+j])
      test_labels.append(0)
  if syn:
    for i in range(fake_sample * 2 // 3,fake_sample):
      for j in range(9):
        syn_fake_features.append(syn_fake[index*fake_sample*9+i*9+j])
        syn_fake_labels.append(0)
  #labels for test syn real
  train_features = np.asarray(train_features).astype('float32')
  test_features = np.asarray(test_features).astype('float32')
  syn_fake_features = np.asarray(syn_fake_features).astype('float32')
  #normalizing to from 0 to 1 by dividing with 255
  train_features = train_features/ 255.
  test_features = test_features / 255.
  syn_fake_features = syn_fake_features /255.
  train_labels = np.array(train_labels)
  test_labels = np.array(test_labels)
  syn_fake_labels = np.array(syn_fake_labels)
  train_labels = to_categorical(train_labels)
  return train_features,train_labels,test_features,test_labels,syn_fake_features,syn_fake_labels

In [ ]:
def make_prediction(data,model):
  tp = model.predict(data)
  output=[]
  for i in tp:
    output.append(i[1])
  return output

In [ ]:
def acc(data,labels):
  output =[]
  for i in data:
    if i>=0.5:
      output.append(1)
    else:
      output.append(0)
  return sklearn.metrics.accuracy_score(output,labels)

In [ ]:
def far_frr_cal(data,compare): #function to calculate FAR and FRR based on 10 thresholds from 0 to 1
  real = 0
  fake = 0
  frr = []
  far =[]
  for i in range (11):
    frr.append(0)
    far.append(0)
  for i in range (len(data)):
    if compare[i]==1:
      real+=1
    else:
      fake+=1
  for i in range (len(data)):
    for j in range(11):
      tp = 1
      if data[i] <= j*0.1:
        tp = 0
      if tp!=compare[i]:
        if compare[i]==0:
          far[j]+=1
        else:
          frr[j]+=1
  for i in range (11):
    frr[i]/=real/100
    far[i]/=fake/100
  return [far,frr]

def far_cal(data,compare):
  far =[]
  for i in range (11):
    far.append(0)
  for i in range (len(data)):
    for j in range(11):
      tp = 1
      if data[i] <= j*0.1:
        tp = 0
      if tp==1:
        far[j]+=1
  for i in range (11):
    far[i]/=len(data)/100
  return far



In [ ]:
def evaluate_person(index):
  y_test = []
  y_label = []
  train_features,train_labels,test_features,test_labels,syn_fake_features,syn_fake_labels = data_generate(index,True)
  batch_size = 32
  epochs = 100
  model = init_model()
  history = model.fit(train_features, train_labels, batch_size=batch_size,epochs=epochs,verbose=1)
  human_test = make_prediction(test_features,model)
  syn_fake_test = make_prediction(syn_fake_features,model)
  y_test.extend(human_test)
  y_test.extend(syn_fake_test)

  y_label.extend(test_labels)
  y_label.extend(syn_fake_labels)
  return acc(human_test,test_labels), acc(syn_fake_test,syn_fake_labels),far_frr_cal(human_test,test_labels),far_cal(syn_fake_test ,syn_fake_labels),human_test,test_labels, y_test, y_label

In [ ]:
y_test = []
y_label = []
human_test = []
human_label = []

In [ ]:
human_frr = []
for i in range (11):
  human_frr.append(0)
human_far = []
for i in range (11):
  human_far.append(0)
syn_real_far = []
for i in range (11):
  syn_real_far.append(0)
syn_fake_far = []
for i in range (11):
  syn_fake_far.append(0)
human_test = []
human_label = []
with open(result_path,'w') as f:
  writer = csv.writer(f)
  writer.writerow(["human_far","human_frr", "synt_fake_far"])
  for i in range (no_contrib):
    print('----------- evaluating person '+str(i+1)+'--------------')
    a,b,x,z, humantest, humanlabel, ytest, ylabel = evaluate_person(i)
    print(x[0][5],x[1][5],z[5])
  f.close()